In [1]:
import torch
import torch.nn as nn
import numpy as np
import copy
import torch.nn.functional as F
import torch.optim as optim
import pickle

import torch
import torch.nn as nn
import numpy as np
import copy
import time
import torch.nn.functional as F
import torch.optim as optim
from itertools import repeat
from tqdm.notebook import tqdm
from collections import defaultdict

In [2]:
import sys
sys.path.append('./SR-GNN-master/pytorch_code')

In [3]:
from model import *
from utils import *

In [60]:
class Opt():
    def __init__(self, ):
        self.dataset = 'yoochoose1_4'
        self.batchSize = 100
        self.hiddenSize = 100
        self.epoch = 30
        self.lr = 0.001
        self.lr_dc = 0.1
        self.lr_dc_step = 3
        self.l2 = 1e-5
        self.step = 1
        self.patience = 10
        self.nonhybrid = False
        self.validation = False
        self.valid_portion = 0.1
        
opt = Opt()

In [61]:
train_data = pickle.load(open('./SR-GNN-master/datasets/' + opt.dataset + '/train.txt', 'rb'))
if opt.validation:
    train_data, valid_data = split_validation(train_data, opt.valid_portion)
    test_data = valid_data
else:
    test_data = pickle.load(open('./SR-GNN-master/datasets/' + opt.dataset + '/test.txt', 'rb'))

if opt.dataset == 'diginetica':
    n_node = 43098
elif opt.dataset == 'yoochoose1_64' or opt.dataset == 'yoochoose1_4':
    n_node = 37484
else:
    n_node = 310

In [62]:
train_seqs = pickle.load(open('./SR-GNN-master/datasets/' + opt.dataset + '/all_train_seq.txt', 'rb'))

In [63]:
set_ = set()
for i in train_data[0]:
    for j in i:
        set_.add(j)
for i in train_data[1]:
    set_.add(i)
item_num = len(set_)

item_mapping = {}
for i, item in enumerate(set_):
    item_mapping[item] = i

n = int((1-opt.valid_portion) * len(train_seqs))
d_basket_train = {k: [item_mapping[item] for item in items] for k, items in enumerate(train_seqs[:-1])}
d_basket_val = {k: [item_mapping[item] for item in items] for k, items in enumerate(train_seqs[1:])}

In [80]:
global_p = 10

In [81]:
#функция тестирования
def test(model, test_data):
    model.eval()
    hit, mrr = [], []
    batch_size = min(opt.batchSize, len(test_data[0]))
    for i in tqdm(range(0, len(test_data[0]), batch_size)):
        scores = model.predict(test_data[0][i:i+batch_size])
        sub_scores = np.array(scores.topk(20)[1])
        targets = test_data[1][i:i+batch_size]
        for score, target in zip(sub_scores, targets):
            target = model.item_mapping.get(target)
            hit.append(np.isin(target, score))
            if len(np.where(score == target)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (np.where(score == target)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100
    return hit, mrr

In [82]:
import torch
import torch.nn as nn
import numpy as np
import copy
import torch.nn.functional as F
import torch.optim as optim
from sklearn.utils.extmath import randomized_svd


class Embedding_prod(nn.Module):
    '''
    Нейросеть для эмбеддинга товара.
    '''
    def __init__(self, data, item_num, item_mapping):
        super(Embedding_prod, self).__init__()
        self.item_mapping = item_mapping
        Dii = defaultdict(int)
        for items in tqdm(train_seqs, total=len(train_seqs)):
            session_len = len(items)
            for i in range(session_len-1):
                Dii[self.item_mapping[items[i]], self.item_mapping[items[i+1]]] += 1
        
        i = torch.LongTensor(list(Dii.keys())).T
        v = torch.LongTensor(list(Dii.values()))
        Q = torch.sparse_coo_tensor(i, v, (item_num+1, item_num+1)).to_dense()
        
        prod_embedd = nn.Embedding(item_num+1, item_num+1).requires_grad_(False)
        prod_embedd.weight = nn.Parameter(Q/(Q.sum(1)[:, None] + 1e-6))
        #prod_embedd.requires_grad = False
        self.prod_embedd = prod_embedd.requires_grad_(False)
        
    def forward(self, prods):
        prod_embedd = self.prod_embedd(prods)
        return prod_embedd


class Embedding_transformer(nn.Module):
    '''
    Нейросеть для эмбеддинга товара.
    '''
    def __init__(self, p=3):
        '''
        На вход:
        X - матрица (товары, эмбеддинг)
        use_cuda - использовать ли cuda
        
        Forward принимает на вход или
        список, содержащий номера товаров,
        или просто номер товара.
        '''
        super(Embedding_transformer, self).__init__()
        self.p = p
        self.sigm = nn.Sigmoid()
        self.base = nn.Parameter(torch.tensor(2.))
        self.params = torch.Tensor(list(range(p))[::-1])
        

    def forward(self, batch):
        t = batch[:, -self.p:, :] / (self.base**self.params[None, :, None])
        return t.sum(1)


class Net(nn.Module):
    def __init__(self, products_embedd, transformer, max_basket_size, item_mapping=item_mapping):
        '''
        На вход:
        products_embedd - объект класса Embedding_prod
        '''
        super(Net, self).__init__()
        self.embedding = products_embedd
        self.transformer = transformer
        self.max_basket_size = max_basket_size
        self.item_mapping = item_mapping
        self.prod_num = len(item_mapping)
    
    def predict(self, baskets):
        rows = []
        range_batch = range(len(baskets))
        for basket in baskets:
            basket_ = [self.item_mapping[item] for item in basket if item in item_mapping]
            padding = [self.prod_num]*(self.max_basket_size - len(basket_) + global_p - 1)
            rows.append(padding+basket_)
        return self.forward(torch.LongTensor(np.array(rows)))
    
    def forward(self, prods):
        embedd = self.embedding(prods)
        return self.transformer(embedd)

In [83]:
max_basket_size = max(map(len, train_seqs))
max_basket_size

200

In [84]:
device = 'cpu'

In [85]:
# создаем сеть
embedd_dim = 1024 # 512
prod_embedd = Embedding_prod(train_seqs, item_num, item_mapping)
transformer = Embedding_transformer(p=global_p)
net_model = Net(prod_embedd, transformer, max_basket_size, item_mapping)
net_model.to(device)

  0%|          | 0/1922630 [00:00<?, ?it/s]

Net(
  (embedding): Embedding_prod(
    (prod_embedd): Embedding(30445, 30445)
  )
  (transformer): Embedding_transformer(
    (sigm): Sigmoid()
  )
)

In [86]:
hit, mrr = test(net_model, test_data)

  0%|          | 0/559 [00:00<?, ?it/s]

In [87]:
hit, mrr

(70.31378582417975, 30.484227748604383)